-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Sort Day Lab

##### Tasks
1. Define a UDF to label the day of week
1. Apply the UDF to label and sort by day of week
1. Plot active users by day of week as a bar graph

In [0]:
%run ../Includes/Classroom-Setup

Python interpreter will be restarted.
Python interpreter will be restarted.



Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03"

Validating the locally installed datasets...(3 seconds)

Predefined tables in "da_sergio_salgado_4613_asp":
  -none-

Predefined paths variables:
  DA.paths.user_db:     dbfs:/mnt/dbacademy-users/sergio.salgado@n.world/apache-spark-programming-with-databricks/database.db
  DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03
  DA.paths.working_dir: dbfs:/mnt/dbacademy-users/sergio.salgado@n.world/apache-spark-programming-with-databricks
  DA.paths.checkpoints: dbfs:/mnt/dbacademy-users/sergio.salgado@n.world/apache-spark-programming-with-databricks/_checkpoints
  DA.paths.sales:       dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03/ecommerce/sales/sales.delta
  DA.paths.users:       dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03/ecommerce/users/users.delta
  DA.paths.events:

Start with a DataFrame of the average number of active users by day of week.

This was the resulting **`df`** in a previous lab.

In [0]:
from pyspark.sql.functions import approx_count_distinct, avg, col, date_format, to_date

df = (spark
      .read
      .format("delta")
      .load(DA.paths.events)
      .withColumn("ts", (col("event_timestamp") / 1e6).cast("timestamp"))
      .withColumn("date", to_date("ts"))
      .groupBy("date").agg(approx_count_distinct("user_id").alias("active_users"))
      .withColumn("day", date_format(col("date"), "E"))
      .groupBy("day").agg(avg(col("active_users")).alias("avg_users"))
     )

display(df)

day,avg_users
Sun,282905.5
Mon,238195.5
Thu,264620.0
Sat,278482.0
Wed,227214.0
Fri,247180.66666666666
Tue,260942.5


### 1. Define UDF to label day of week

Use the **`label_day_of_week`** function provided below to create the UDF **`label_dow_udf`**

In [0]:
def label_day_of_week(day: str) -> str:
    dow = {"Mon": "1", "Tue": "2", "Wed": "3", "Thu": "4",
           "Fri": "5", "Sat": "6", "Sun": "7"}
    return dow.get(day) + "-" + day

In [0]:
# TODO
label_dow_udf = spark.udf.register("label_dow", label_day_of_week)

### 2. Apply UDF to label and sort by day of week
- Update the **`day`** column by applying the UDF and replacing this column
- Sort by **`day`**
- Plot as a bar graph

In [0]:
# TODO
final_df = (df
           .withColumn("day", label_dow_udf(col("day")))
           .sort("day")
           )

display(final_df)

day,avg_users
1-Mon,238195.5
2-Tue,260942.5
3-Wed,227214.0
4-Thu,264620.0
5-Fri,247180.66666666666
6-Sat,278482.0
7-Sun,282905.5


### Clean up classroom

In [0]:
DA.cleanup()

Resetting the learning environment...
...dropping the database "da_sergio_salgado_4613_asp"...(1 seconds)
...removing the working directory "dbfs:/mnt/dbacademy-users/sergio.salgado@n.world/apache-spark-programming-with-databricks"...(0 seconds)

Validating the locally installed datasets...(4 seconds)


-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>